In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [7]:
data = pd.read_csv('../data/demo_data.csv', index_col=[0])

In [9]:
set(data['if_0'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

In [2]:
# parse configuration file
with open('../config/feat_info.json', 'r') as r:
    info = json.load(r)

coltype = info['colType']
encode_info = info['encode']
bin_info = info['manualBox']

# load data
data = pd.read_csv('../data/demo_data.csv', index_col=[0])


feat_names = list(data.columns)

# parse feature
for col_name in feat_names:
    if col_name in coltype:
        # binning of continuous feature values
        if coltype[col_name] == 'M':
            bins = bin_info[col_name]
            try:
                # 分箱正常是从1开始，碰到异常置为0
                data[col_name] = np.digitize(data[col_name], bins=bins) + 1
            except Exception as e:
                print(e)
                data[col_name] = 0
        # encoding categorical features
        elif coltype[col_name] == 'E':
            code_dict = encode_info[col_name]
            try:
                # encoding starts from 1, with anomalies set to 0
                data[col_name] = data[col_name].apply(lambda x: code_dict[str(int(float(x)))])
            except Exception as e:
                print(e)
                data[col_name] = 0
    else:
        # not process
        pass


In [3]:
data

,uf_0,uf_1,uf_2,uf_3,uf_4,if_0,uf_5,uf_6,uf_7,uf_8,uf_9,uf_10,uf_11,uf_12,uf_13,uf_14,uf_15,uf_16,label
0,4,1,1,7,1,1,1,4,2,3,3,1,2,3,10,9,9,10,0
1,1,1,1,1,1,5,1,4,3,3,6,11,2,2,3,9,6,7,0
2,1,1,1,1,1,12,1,5,2,4,3,1,2,2,11,9,3,10,0
3,4,1,1,10,1,2,1,3,3,4,5,2,2,6,3,3,3,8,0
4,4,1,2,12,1,1,4,4,7,3,3,5,3,3,5,3,8,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,1,2,13,1,5,2,3,2,3,1,4,2,3,10,7,1,4,0
9996,1,1,1,1,1,2,1,4,2,3,1,1,2,2,11,9,1,2,0
9997,1,1,1,1,1,12,1,1,1,1,1,1,1,1,1,1,1,1,0
9998,1,1,3,11,1,8,4,3,2,3,8,2,2,12,3,9,9,10,0


In [4]:
# split the dataset into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [5]:
test_data = test_data.reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

In [6]:
train_data.to_csv('../data/train_data_fornn.csv')
test_data.to_csv('../data/test_data_fornn.csv')